In [10]:
from sklearn.metrics import *
from tqdm import tqdm
import numpy as np
import pyvista as pv
import open3d as o3d
import shap
from sklearn.cluster import KMeans

from final_models_explainability.get_predictions import *

In [27]:
results_raw = np.load('volumes_eval.npz', allow_pickle=True)
results = {key: results_raw[key] for key in results_raw.files}

correct_indices = []

for i in range(len(results['true'])):

    if results['true'][i] == 1 and results['pred_classes'][i] == 1:

        correct_indices.append(i)

results['shap_values_list'] = results['shap_values_list'].tolist()

filtered_results = {
                    'true': [results['true'][i] for i in correct_indices],
                    'pred_probs': [results['pred_probs'][i] for i in correct_indices],
                    'data': [results['data'][i] for i in correct_indices],
                    'shap_values_list': [results['shap_values_list'][i] for i in correct_indices],
                }

In [ ]:
with open('final_models_explainability/volumes_gbdt.pkl', "rb") as f:
    model = pickle.load(f)

# Arbitrary subject for struct names
subject = Subject("mri_samples/chris_t1", None)

explainer = shap.Explainer(model, np.array(subject.aseg_stats['StructName']))

shap_values = explainer(filtered_results['data'])

print(len(shap_values))

shap.plots.bar(shap_values[0], max_display=10)

shap.summary_plot(shap_values, filtered_results['data'])

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: HistGradientBoostingClassifier(class_weight='balanced', l2_regularization=0,
                               learning_rate=0.5, max_features=0.5,
                               max_leaf_nodes=30)